In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense , GlobalAveragePooling2D , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD , Adam
from tensorflow.keras.callbacks import CSVLogger
import pandas as pd
from sklearn.metrics import classification_report
import os
import warnings
warnings.filterwarnings( 'ignore' )

In [3]:
train_dir = '/content/drive/MyDrive/BACHAugment'
test_dir = '/content/drive/MyDrive/BACHtest'

# Define the image size and batch size
image_size = (224, 224)
batch_size = 32

random_seed = np.random.seed(1142)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= image_size,
    batch_size=batch_size,
    seed = random_seed,
    shuffle=False,
    subset = 'training',
    class_mode='categorical')

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= image_size,
    batch_size=batch_size,
    seed = random_seed,
    shuffle=False,
    subset = 'validation',
    class_mode='categorical')

test_datagen=ImageDataGenerator(rescale=1./255)
test_generator =test_datagen.flow_from_directory(test_dir,
                                                      target_size=image_size,
                                                          batch_size=batch_size,
                                                          seed=random_seed,
                                                          shuffle=False,
                                                          class_mode='categorical') # set as training data

Found 3368 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
Found 103 images belonging to 4 classes.


In [6]:
base_model = EfficientNetB3(input_shape=(224,244,3), include_top=False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


optimizer = SGD(learning_rate=1e-3)
# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

43941136/43941136 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 244, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 244, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 244, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)     

In [ ]:
csv_logger = CSVLogger('/content/drive/MyDrive/BACH-Results/EfficientNet/training_results.csv')

# Train the model
history = model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[csv_logger])

Epoch 1/100
106/106 [==============================] - 3601s 34s/step - loss: 1.4349 - accuracy: 0.2429 - val_loss: 1.3978 - val_accuracy: 0.2131
Epoch 2/100
106/106 [==============================] - 77s 725ms/step - loss: 1.4238 - accuracy: 0.2438 - val_loss: 1.3941 - val_accuracy: 0.2595
Epoch 3/100
106/106 [==============================] - 87s 819ms/step - loss: 1.4192 - accuracy: 0.2548 - val_loss: 1.4014 - val_accuracy: 0.2429
Epoch 4/100
106/106 [==============================] - 79s 745ms/step - loss: 1.4134 - accuracy: 0.2533 - val_loss: 1.3718 - val_accuracy: 0.2786
Epoch 5/100
106/106 [==============================] - 79s 748ms/step - loss: 1.4067 - accuracy: 0.2681 - val_loss: 1.4078 - val_accuracy: 0.2524
Epoch 6/100
106/106 [==============================] - 79s 738ms/step - loss: 1.3996 - accuracy: 0.2729 - val_loss: 1.4240 - val_accuracy: 0.2226
Epoch 7/100
106/106 [==============================] - 77s 720ms/step - loss: 1.3961 - accuracy: 0.2729 - val_loss: 1.4205 -

In [ ]:
import pickle

# Save model weights
model.save_weights('/content/drive/MyDrive/BACH-Results/6.h5')

# Save training history
with open('/content/drive/MyDrive/BACH-Results/VGG16/vgg_16training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)